# CS 109 Extension Project

Part 1: Get All Necessary Data

In [1]:
# Goal: Get Live Feed Data
import requests
import os
import time

# make a directory for the data
download_directory = "data"
# create the directory if it doesn't exist
os.makedirs(download_directory, exist_ok=True)

# mark how many days in each month
days = {2: 28, 4: 30, 6: 30, 9: 30, 11: 30}

# loop through each day
for i in range(1, 13):
    day = 0
    if i in days:
        day = days[i]
    else:
        day = 31

    for j in range(1, day + 1):
        # get associated file name
        file_name = f"subwaydatanyc_2023-{i:02}-{j:02}_csv.tar.xz"
        full_path = os.path.join(download_directory, file_name)

        # pull the data from the URL
        url = f"https://subwaydata.nyc/data/{file_name}"

        try:
            # query for response
            response = requests.get(url)

            # if successful, write data to laptop
            if response.status_code == 200:
                with open(full_path, "wb") as f:
                    f.write(response.content)
            # catch failures
            else:
                print(f"Error {response.status_code} accessing: {file_name}")

        except requests.exceptions.RequestException as e:
            print(f"Connection Error for {file_name}: {e}")
            time.sleep(5) # Wait longer after a connection error

/Users/leomelton/PyCharmMiscProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [35]:
# extract from .tar.xz file extension
import tarfile
import os

# define the necessary file paths
data_directory = "data"
extract_dir = 'extracted_csv_data'
os.makedirs(extract_dir, exist_ok=True)

for i in range(1, 13):
    day = 0
    if i in days:
        day = days[i]
    else:
        day = 31

    for j in range(1, day + 1):
        # define files to grab
        archive_file = os.path.join(data_directory, f'subwaydatanyc_2023-{i:02}-{j:02}_csv.tar.xz')
        target_file_in_archive = f'subwaydatanyc_2023-{i:02}-{j:02}_stop_times.csv'

        # open the archive, get just the stop_times.csv file
        try:
            print(f"Starting extraction of ONLY {target_file_in_archive} from {archive_file}...")

            # read and decompress the file
            with tarfile.open(archive_file, 'r:xz') as tar:

                # check for file
                if target_file_in_archive in tar.getnames():
                    # extract to extracted_csv_data
                    tar.extract(target_file_in_archive, path=extract_dir)

                    # create full path
                    extracted_path = os.path.join(extract_dir, target_file_in_archive)

        # catch errors
        except tarfile.ReadError:
            print(f"\n❌ Error: Could not read or decompress the archive '{archive_file}'. Check file integrity.")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}")

Starting extraction of ONLY subwaydatanyc_2023-01-01_stop_times.csv from data/subwaydatanyc_2023-01-01_csv.tar.xz...
Starting extraction of ONLY subwaydatanyc_2023-01-02_stop_times.csv from data/subwaydatanyc_2023-01-02_csv.tar.xz...
Starting extraction of ONLY subwaydatanyc_2023-01-03_stop_times.csv from data/subwaydatanyc_2023-01-03_csv.tar.xz...
Starting extraction of ONLY subwaydatanyc_2023-01-04_stop_times.csv from data/subwaydatanyc_2023-01-04_csv.tar.xz...
Starting extraction of ONLY subwaydatanyc_2023-01-05_stop_times.csv from data/subwaydatanyc_2023-01-05_csv.tar.xz...
Starting extraction of ONLY subwaydatanyc_2023-01-06_stop_times.csv from data/subwaydatanyc_2023-01-06_csv.tar.xz...
Starting extraction of ONLY subwaydatanyc_2023-01-07_stop_times.csv from data/subwaydatanyc_2023-01-07_csv.tar.xz...
Starting extraction of ONLY subwaydatanyc_2023-01-08_stop_times.csv from data/subwaydatanyc_2023-01-08_csv.tar.xz...
Starting extraction of ONLY subwaydatanyc_2023-01-09_stop_times.

In [4]:
# Gather Data from CSV Files
import pandas as pd

days = {2: 28, 4: 30, 6: 30, 9: 30, 11: 30}

# Function to classify a piece of data as "Rush Hour", "Weekends", or "Nights" for use for hour slicing
# given a timestamp.
def categorize_time(timestamp):

    # get a day of the week
    day = timestamp.weekday()
    hour = timestamp.hour

    # Nights (10 p.m. to 6 a.m. EVERY DAY)
    # window split into two chunks: (10 PM - 11:59 PM) OR (12 AM - 5:59 AM)
    is_night = (hour >= 22) or (hour < 6)
    if is_night:
        return 'Nights'

    # Weekends (Saturday=5, Sunday=6 in datetime)
    is_weekend = (day >= 5)
    if is_weekend:
        return 'Weekends'

    # Rush Hour (6 a.m. to 10 a.m. OR 4 p.m. to 8 p.m. on Weekdays)
    is_am_rush = (hour >= 6) and (hour < 10)
    is_pm_rush = (hour >= 16) and (hour < 20)

    # AM or PM rush is "Rush Hour" classification
    if is_am_rush or is_pm_rush:
        return 'Rush Hour'

    # Last category, not used in analysis
    return 'Off-Peak Weekday'

# Create dicts to hold the three categories for F train and generally
f_delays_by_category = {'Rush Hour': [], 'Weekends': [], 'Nights': []}
all_delays_by_category = {'Rush Hour': [], 'Weekends': [], 'Nights': []}

# Arrays to hold: all delays, delays on the F train, and delays on the F train with an additional slice
# for stop name
all_delay = []
f_delay = []
f_delay_by_stop = []

# again, loop by day (each day has one CSV file)
for i in range(1, 13):
    day = 0
    if i in days:
        day = days[i]
    else:
        day = 31

    for j in range(1, day + 1):
        # read in CSV
        df = pd.read_csv(f"extracted_csv_data/subwaydatanyc_2023-{i:02}-{j:02}_stop_times.csv")

        # get next departure
        df['next_actual_departure'] = df.groupby('trip_uid')['marked_past'].shift(-1)

        # calculate trip duration between stops
        df['actual_duration_seconds'] = (
            df['next_actual_departure'] - df['marked_past']
        )

        # get next expected departure
        df['next_scheduled_departure'] = df['departure_time'].shift(-1)

        # get expected length of trip
        df['expected_duration_seconds'] = (
            df['next_scheduled_departure'] - df['departure_time']
        )

        # filter out trips that are too short (likely erroneous data)
        df_filtered = df[df['actual_duration_seconds'] > (df['expected_duration_seconds'] * 0.5)]
        df_filtered = df_filtered[df_filtered['actual_duration_seconds'] > 30]

        # calculate delay added
        df_filtered['delay_added_seconds'] = (
            df_filtered['actual_duration_seconds'] - df_filtered['expected_duration_seconds']
        )

        # remove faulty NaN data
        df_final = df_filtered.dropna(subset=['delay_added_seconds'])

        # Data filtering to remove some anomalies in the data set
        # take out data of trains that start really late
        df_internal_run = (
            df_final.groupby('trip_uid')
            .tail(-1)
        )
        # toss all data over 10800 seconds (3 hour delay), too egregious to include in data
        MAX_DELAY_SECONDS = 10800
        df_internal_run = df_internal_run[
            df_internal_run['delay_added_seconds'] <= MAX_DELAY_SECONDS
        ].copy()
        # take out stops in Staten Island that are not being considered
        df_subway_only = df_internal_run[~df_internal_run['stop_id'].str.startswith('S')].copy()

        # Take the CSV data and add it to respective files

        # Filter for F trains and put it into its own df
        df_f_train = df_subway_only[
            df_subway_only['trip_uid'].astype(str).str.contains(r'_F\.\.', regex=True)
        ].copy()

        # place into respective arrays
        all_delay.append(df_subway_only['delay_added_seconds']) # all train delays
        f_delay.append(df_f_train['delay_added_seconds']) # F train delay
        f_delay_by_stop.append(df_f_train[['stop_id', 'delay_added_seconds']]) # F train, including stop slice

        # convert Unix time from CSV files to datetime
        df_f_train['datetime_local'] = pd.to_datetime(
            df_f_train['marked_past'],
            unit='s',
            errors='coerce'
        )
        df_subway_only['datetime_local'] = pd.to_datetime(
            df_subway_only['marked_past'],
            unit='s',
            errors='coerce'
        )

        # again, drop rows where conversion fails
        df_f_train.dropna(subset=['datetime_local'], inplace=True)
        df_subway_only.dropna(subset=['datetime_local'], inplace=True)


        # apply categorization function to data
        df_f_train['time_category'] = df_f_train['datetime_local'].apply(categorize_time)
        df_subway_only['time_category'] = df_subway_only['datetime_local'].apply(categorize_time)

        # slice by category
        for category in ['Rush Hour', 'Weekends', 'Nights']:

            # add F-train data
            df_f_cat = df_f_train[df_f_train['time_category'] == category]
            if not df_f_cat.empty:
                f_delays_by_category[category].append(df_f_cat['delay_added_seconds'])

            # add system data
            df_all_cat = df_subway_only[df_subway_only['time_category'] == category]
            if not df_all_cat.empty:
                all_delays_by_category[category].append(df_all_cat['delay_added_seconds'])

# Previous Experimental Code to Gather per-Route Info
#         total_average = df_subway_only['delay_added_seconds'].mean()
#         route_data['total'].append(total_average)
#
#         for route in trains:
#             route_df = df_subway_only[
#                 df_subway_only['trip_uid'].str.contains(fr'_{route}\.\.', regex=True)
#             ].copy()
#
#             route_avg = route_df['delay_added_seconds'].mean()
#             route_data[route].append(route_avg)
#
# for route in route_data:
#     average = sum(route_data[route]) / len(route_data[route])
#     print("Average of " + route + " is " + str(average))

Initial Analysis: Overall Dataset

In [2]:
# Construct PMF
# concatenate data together
all_delays_series = pd.concat(all_delay, ignore_index=True)
f_delays_series = pd.concat(f_delay, ignore_index=True)

# imports
import numpy as np

# chop-off extremes (more than 5 minutes early and more than 20 minutes late) and
# mark them as -300 sec and 1200 sec respectively to go into buckets
modified_f_del = [1200 if x > 1200 else -300 if x < -300 else x for x in f_delays_series]
modified_all_del = [1200 if x > 1200 else -300 if x < -300 else x for x in all_delays_series]

# Construct bins for histogram
min_delay = -300
max_delay = 1200
bin_width = 60
bins = np.arange(
    start=np.floor(min_delay / bin_width) * bin_width,
    stop=np.ceil(max_delay / bin_width) * bin_width + bin_width,
    step=bin_width
)
print(bins)

# Construct histogram using data
hist_total, _ = np.histogram(modified_all_del, bins=bins, density=True)
hist_f, _ = np.histogram(modified_f_del, bins=bins, density=True)


NameError: name 'all_delay' is not defined

In [39]:
# Plot the Initial Distributions
import matplotlib.pyplot as plt

# Calculate center of bins
bin_centers = (bins[:-1] + bins[1:]) / 2

# Calculate width of bars
bar_width = bins[1] - bins[0]

# Create the plot
plt.figure(figsize=(12, 6))

# Plot the F Train PMF
plt.bar(
    bin_centers,
    hist_f,
    width=bar_width * 0.9,
    alpha=0.6,
    label='F Train',
    color='blue'
)

# Plot the Overall System PMF
plt.bar(
    bin_centers,
    hist_total,
    width=bar_width * 0.9,
    alpha=0.6,
    label='Overall System',
    color='red'
)

# Set labels and title
plt.title('Distribution of Added Delay: F Train vs. System Average')
plt.xlabel(f'Added Delay (seconds), Bins of {bar_width} seconds')
plt.ylabel('Probability (PMF)')
plt.legend()
plt.grid(axis='y', alpha=0.5)

# Logarithmic scale
plt.yscale('log')

# Save the figure
plt.savefig('delay_distribution_comparison.png') # <- this is included in the repo for your viewing
plt.close()

In [30]:
# Apply bootstrapping on the dataset to get the KL Divergence
TRIALS = 1000
N_F = len(f_delays_series)
N_TOT = len(all_delays_series)

# Function to calculate KL Divergence (with LaPlace Smoothing, extra feature to prevent division by 0)
def calculate_kl_divergence(P, Q, epsilon=1e-12):
    # apply laplace smoothing
    P_smoothed = (P + epsilon) / (np.sum(P) + len(P) * epsilon)
    Q_smoothed = (Q + epsilon) / (np.sum(Q) + len(Q) * epsilon)

    # calculate KL divergence
    kl_divergence = np.sum(P_smoothed * np.log(P_smoothed / Q_smoothed))

    return kl_divergence

# init var
divergences = []
# loop 1000 times
for i in range(TRIALS):
    # get
    f_sample = f_delays_series.sample(n=N_F, replace=True)
    tot_sample = all_delays_series.sample(n=N_TOT, replace=True)

    # 2. PMF GENERATION: Re-calculate the PMFs for the new samples
    # Density=True normalizes the histogram to create the PMF array.
    sample_f_hist, _ = np.histogram(f_sample, bins=bins, density=True)
    sample_tot_hist, _ = np.histogram(tot_sample, bins=bins, density=True)

    # 3. KL CALCULATION: Calculate the divergence
    kl_score = calculate_kl_divergence(sample_f_hist, sample_tot_hist)

    # 4. Store the result
    divergences.append(kl_score)

    if (i + 1) % 100 == 0:
        print(f"Completed {i + 1} iterations.")

# convert to numpy and output mean
kl_distribution = np.array(divergences)
kl_mean = np.sum(kl_distribution) / TRIALS
print(kl_mean)

Completed 100 iterations.
Completed 200 iterations.
Completed 300 iterations.
Completed 400 iterations.
Completed 500 iterations.
Completed 600 iterations.
Completed 700 iterations.
Completed 800 iterations.
Completed 900 iterations.
Completed 1000 iterations.
0.03571832296035114


In [60]:
# Check statistics of the sampled KL divergences

# find standard deviation
kl_std = np.std(kl_distribution)
print(kl_std)

# Find lengths of our input datas
print(len(f_delays_series))
print(len(all_delays_series))

8.933949310066523e-05
4561185
60425425


Analysis by Time of Day

In [6]:
import numpy as np

# aggregate all datasets
f_delays_rush_series = pd.concat(f_delays_by_category['Rush Hour'], ignore_index=True)
all_delays_rush_series = pd.concat(all_delays_by_category['Rush Hour'], ignore_index=True)

f_delays_weekend_series = pd.concat(f_delays_by_category['Weekends'], ignore_index=True)
all_delays_weekend_series = pd.concat(all_delays_by_category['Weekends'], ignore_index=True)

f_delays_nights_series = pd.concat(f_delays_by_category['Nights'], ignore_index=True)
all_delays_nights_series = pd.concat(all_delays_by_category['Nights'], ignore_index=True)

# like before, define bins
min_delay = -300
max_delay = 1200
bin_width = 60
bins = np.arange(
    start=min_delay,
    stop=max_delay + bin_width,
    step=bin_width
)

# calculate the PMFs for each
hist_f_rush, _ = np.histogram(f_delays_rush_series, bins=bins, density=True)
hist_all_rush, _ = np.histogram(all_delays_rush_series, bins=bins, density=True)

hist_f_weekend, _ = np.histogram(f_delays_weekend_series, bins=bins, density=True)
hist_all_weekend, _ = np.histogram(all_delays_weekend_series, bins=bins, density=True)

hist_f_nights, _ = np.histogram(f_delays_nights_series, bins=bins, density=True)
hist_all_nights, _ = np.histogram(all_delays_nights_series, bins=bins, density=True)

print("Six final PMF variables have been created for KL-Divergence calculation:")
print("Rush Hour: hist_f_rush, hist_all_rush")
print("Weekends: hist_f_weekend, hist_all_weekend")
print("Nights: hist_f_nights, hist_all_nights")

Six final PMF variables have been created for KL-Divergence calculation:
Rush Hour: hist_f_rush, hist_all_rush
Weekends: hist_f_weekend, hist_all_weekend
Nights: hist_f_nights, hist_all_nights


In [73]:
# Initial analysis: look at the average delay for each. We notice that they're all pretty similar,
# like before
print(np.mean(f_delays_rush_series))
print(np.mean(all_delays_rush_series))

print(np.mean(f_delays_weekend_series))
print(np.mean(all_delays_weekend_series))

print(np.mean(f_delays_nights_series))
print(np.mean(all_delays_nights_series))

0.40807885170994407
2.5933215325333667
0.39649521594333137
2.878245985337892
0.373835324522228
2.485268401585042


In [7]:
from scipy.special import rel_entr
import numpy as np
import pandas as pd # Needed for sampling

# Helper functions
# Calculate smoothed KL, again using Laplace smoothing
def calculate_smoothed_kl(P, Q, epsilon=1e-12):
    P_smoothed = (P + epsilon) / (np.sum(P) + len(P) * epsilon)
    Q_smoothed = (Q + epsilon) / (np.sum(Q) + len(Q) * epsilon)

    return np.sum(P_smoothed * np.log(P_smoothed / Q_smoothed))


N_BOOTSTRAPS = 500  # 500 trials
results = {'Rush Hour': [], 'Weekends': [], 'Nights': []}
categories = ['Rush Hour', 'Weekends', 'Nights']

# map names to variables
data_map = {
    'Rush Hour': (f_delays_rush_series, all_delays_rush_series),
    'Weekends': (f_delays_weekend_series, all_delays_weekend_series),
    'Nights': (f_delays_nights_series, all_delays_nights_series),
}

# begin bootstrap
print(f"Starting {N_BOOTSTRAPS} bootstrap iterations...")

for i in range(N_BOOTSTRAPS):

    # Resample all six series in one go
    resampled_data = {}
    # for each category
    for cat in categories:
        P_series, Q_series = data_map[cat]

        # get number of required samples
        P_sample = P_series.sample(n=len(P_series), replace=True)
        Q_sample = Q_series.sample(n=len(Q_series), replace=True)

        # recalculate PMFs
        P_hist, _ = np.histogram(P_sample, bins=bins, density=True)
        Q_hist, _ = np.histogram(Q_sample, bins=bins, density=True)

        # calculate KL and append
        kl_score = calculate_smoothed_kl(P_hist, Q_hist)
        results[cat].append(kl_score)

print("Bootstrapping complete.")

# --- 4. Calculate and Print Confidence Intervals ---

print("\n=================================================")
print("  KL-Divergence Results")
print("=================================================")

for cat in categories:
    scores = np.array(results[cat])
    # The 95% CI is defined by the 2.5th and 97.5th percentiles
    ci_low = np.percentile(scores, 2.5)
    ci_high = np.percentile(scores, 97.5)

    print(f"KL({cat}): Mean={np.mean(scores):.5f}, Std={np.std(scores):.5f}")

Starting 500 bootstrap iterations...
Bootstrapping complete.

  KL-Divergence Results
KL(Rush Hour): Mean=0.03544, Std=0.00019
KL(Weekends): Mean=0.03620, Std=0.00027
KL(Nights): Mean=0.03554, Std=0.00015


Analysis By Stop on the F Train

In [53]:
# Analyze by stop

# Group data into one large array
f_stops_full = pd.concat(f_delay_by_stop, ignore_index=True)
f_stops_full['station_id'] = f_stops_full['stop_id'].str[:-1]

# Group by stop ID and calculate delay
segment_performance = f_stops_full.groupby('station_id')['delay_added_seconds'].agg(
    ['mean', 'count']
).reset_index()

# Sort to find worst segments
min_count_threshold = 1000 # require at least 1,000 observations to pull out anomalies
segment_performance_filtered = segment_performance[
    segment_performance['count'] >= min_count_threshold
]

# Find worst bottlenecks
worst_segments = segment_performance_filtered.sort_values(
    by='mean',
    ascending=False
).head(10)

# output
print(f"\n--- Top 10 F-Train Bottlenecks (Segments with Highest Added Delay) ---")
print(worst_segments)


--- Top 10 F-Train Bottlenecks (Segments with Highest Added Delay) ---
    station_id      mean   count
111        G09  1.803525   20257
29         B04  1.550775   69453
117        G15  1.444933   19313
33         B10  0.929718   69577
116        G14  0.842029  110286
110        G08  0.740248  110667
32         B08  0.733398   69947
82         F09  0.699076   40389
115        G13  0.670380   20196
114        G12  0.610227   20181


In [59]:
# Prettier Table of  Top 10 Most Delayed Stops on the F
import numpy as np

# Mappings for 10 most frequent stop IDs to their real names
stop_id_to_name = {
    'G09': "67 Av (Q)",
    'B04': '21 St-Queensbridge (Q)',
    'G15': '65 St (Q)',
    'B10': '57 St (M)',
    'G14': 'Jackson Hts-Roosevelt Av (Q)',
    'G08': 'Forest Hills-71 Av (Q)',
    'B08': 'Lexington Ave/63 St (Q)',
    'F09': 'Court Sq-23 St (Q)',
    'G13': 'Elmhurst Av (Q)',
    'G12': 'Grand Av-Newtown (Q)',
}

# add a column that shows the real name of each stop
segment_performance_filtered['station_name'] = segment_performance_filtered['station_id'].map(stop_id_to_name)

# sort by most delayed on average
worst_segments = segment_performance_filtered.sort_values(
    by='mean',
    ascending=False
).head(10)

# Create Chart
print(f"\n--- Top 10 F-Train Bottlenecks (Segments with Highest Added Delay) ---")
# Print the stop_id, the new station_name, mean, and count
print(worst_segments[['station_id', 'station_name', 'mean', 'count']])



--- Top 10 F-Train Bottlenecks (Segments with Highest Added Delay) ---
    station_id                  station_name      mean   count
111        G09                     67 Av (Q)  1.803525   20257
29         B04        21 St-Queensbridge (Q)  1.550775   69453
117        G15                     65 St (Q)  1.444933   19313
33         B10                     57 St (M)  0.929718   69577
116        G14  Jackson Hts-Roosevelt Av (Q)  0.842029  110286
110        G08        Forest Hills-71 Av (Q)  0.740248  110667
32         B08       Lexington Ave/63 St (Q)  0.733398   69947
82         F09            Court Sq-23 St (Q)  0.699076   40389
115        G13               Elmhurst Av (Q)  0.670380   20196
114        G12          Grand Av-Newtown (Q)  0.610227   20181


/var/folders/6k/xht3dk1x1jd5s5_8h_ljjfc40000gn/T/ipykernel_45171/978103061.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segment_performance_filtered['station_name'] = segment_performance_filtered['station_id'].map(stop_id_to_name)


# Miscellaneous Tests and Experiments

In [32]:
# Test 1: Analyze One File for Functionality
import pandas as pd

# read CSV
df = pd.read_csv("extracted_csv_data/subwaydatanyc_2023-01-01_stop_times.csv")

# group by trip (one train's journey), and align departure time
df['next_actual_departure'] = df.groupby('trip_uid')['marked_past'].shift(-1)

# get actual duration from stop to stop
df['actual_duration_seconds'] = (
    df['next_actual_departure'] - df['marked_past']
)

# get the next scheduled departure and calculate expected duration
df['next_scheduled_departure'] = df['departure_time'].shift(-1)
df['expected_duration_seconds'] = (
    df['next_scheduled_departure'] - df['departure_time']
)

# filter out for trips that are longer than 30 seconds and are relatively close to expected trip time
df_filtered = df[df['actual_duration_seconds'] > (df['expected_duration_seconds'] * 0.5)]
df_filtered = df_filtered[df_filtered['actual_duration_seconds'] > 30]

# calculate the delay added/stop
df_filtered['delay_added_seconds'] = (
    df_filtered['actual_duration_seconds'] - df_filtered['expected_duration_seconds']
)

# remove any stops with NaN (last stop of each trip)
df_final = df_filtered.dropna(subset=['delay_added_seconds'])

        expected_duration_seconds  delay_added_seconds
0                           150.0                  0.0
1                           109.0                -19.0
2                            79.0                 11.0
3                            76.0                 -1.0
4                            81.0                  0.0
...                           ...                  ...
155523                      213.0                 -3.0
155524                      184.0                -73.0
155548                      160.0                -73.0
155572                      120.0                  0.0
155573                       66.0                 -4.0

[130929 rows x 2 columns]
130929
                 trip_uid stop_id  delay_added_seconds
18888     1672576650_2..N    123N               -226.0
41870  1672601580_2..S01R    132S               -224.0
36097  1672594920_3..N01R    120N               -215.0
18787  1672576590_4..S06R    235S               -200.0
39875  1672599240_3..N01R    12

In [34]:
# Test 2: Miscellaneous Analysis on Train Data (organized by route)

total_average = df_final['delay_added_seconds'].mean()

df_subway_only = df_final[~df_final['stop_id'].str.startswith('S')].copy()
df_f_train = df_subway_only[
    df_subway_only['trip_uid'].astype(str).str.contains('F')
].copy()

two_train_df = df_final[
    df_final['trip_uid'].str.contains(r'_2\.\.', regex=True)
].copy()

six_train_df = df_final[
    df_final['trip_uid'].str.contains(r'_6\.\.', regex=True)
].copy()

two_avg = two_train_df['delay_added_seconds'].mean()
six_avg = six_train_df['delay_added_seconds'].mean()
F_average = df_f_train['delay_added_seconds'].mean()

print(F_average)
print(two_avg)
print(six_avg)
print(total_average)

-0.006378189094547274
5.210709504685409
1.9174973488865323
2.0375928938585033
